In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import csv
import os
import tensorflow as tf
from scipy import misc

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def convolution(img, f, w=1):
    img_copy = np.copy(img)
    size_x = img_copy.shape[0]
    size_y = img_copy.shape[1]
    for x in range(1,size_x-1):
        for y in range(1,size_y-1):
            convolution = 0.0
            convolution = convolution + (img_copy[x - 1, y-1] * f[0][0])
            convolution = convolution + (img_copy[x, y-1] * f[0][1])
            convolution = convolution + (img_copy[x + 1, y-1] * f[0][2])
            convolution = convolution + (img_copy[x-1, y] * f[1][0])
            convolution = convolution + (img_copy[x, y] * f[1][1])
            convolution = convolution + (img_copy[x+1, y] * f[1][2])
            convolution = convolution + (img_copy[x-1, y+1] * f[2][0])
            convolution = convolution + (img_copy[x, y+1] * f[2][1])
            convolution = convolution + (img_copy[x+1, y+1] * f[2][2])
            convolution = convolution * w
            if(convolution<0):
                convolution=0
            if(convolution>255):
                convolution=255
            img[x, y] = convolution

def max_pooling(img):
    size_x = img.shape[0]
    size_y = img.shape[1]
    new_x = int(size_x/2)
    new_y = int(size_y/2)
    new_img = np.zeros((new_x, new_y))
    for x in range(0, size_x, 2):
        for y in range(0, size_y, 2):
            pixels = []
            pixels.append(img[x, y])
            pixels.append(img[x+1, y])
            pixels.append(img[x, y+1])
            pixels.append(img[x+1, y+1])
            new_img[int(x/2),int(y/2)] = max(pixels)
    return new_img

def avg_pooling(img):
    size_x = img.shape[0]
    size_y = img.shape[1]
    new_x = int(size_x/2)
    new_y = int(size_y/2)
    new_img = np.zeros((new_x, new_y))
    for x in range(0, size_x, 2):
        for y in range(0, size_y, 2):
            pixels = []
            pixels.append(img[x, y])
            pixels.append(img[x+1, y])
            pixels.append(img[x, y+1])
            pixels.append(img[x+1, y+1])
            new_img[int(x/2),int(y/2)] = sum(pixels) / len(pixels)
    return new_img

def demonstrate_conv_and_pooling(f):
    img = misc.ascent()
    fig = plt.figure(figsize=(20,6))
    ax = fig.add_subplot(1,4,1)
    ax.set_title('Original')
    ax.imshow(img)
    plt.legend()
    ax = fig.add_subplot(1,4,2)
    convolution(img, f)
    ax.imshow(img)
    ax.set_title('Convolution')
    plt.legend()
    ax = fig.add_subplot(1,4,3)
    ax.imshow(max_pooling(img))
    ax.set_title('Max pooling')
    plt.legend()
    ax = fig.add_subplot(1,4,4)
    ax.imshow(avg_pooling(img))
    ax.set_title('Avg pooling')
    plt.legend()
    plt.show()

In [ ]:
#Experiment with different values for fun effects.
#f = [ [0, 1, 0], [1, -4, 1], [0, 1, 0]]
#f = [ [1, 2, 3], [-1, -2, -3], [0, 1, 0]]
#f = [ [0, 1, 2], [-1, 0, 1], [-2, -1, 0]]
f = [ [-1, -2, -1], [0, 0, 0], [1, 2, 1]]
#f = [ [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]

plt.gray()

demonstrate_conv_and_pooling(f)

In [ ]:
def get_data(filename):
    with open(filename) as file:        
        csv_reader = csv.reader(file)  
        next(csv_reader)  
        
        images = []
        labels = []
        
        for row in csv_reader:
            images.append(np.array(row[1:]).reshape(28, 28))
            labels.append(row[0])

    images = np.array(images).astype(np.float32) 
    labels = np.array(labels).astype(np.float32)
    
    return images, labels

In [ ]:
mnist_train = "/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
mnist_test = "/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv"
training_images, training_labels = get_data(mnist_train)
testing_images, testing_labels = get_data(mnist_test)

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

In [ ]:
def plot_data():
    imgs = np.random.randint(training_images.shape[0], size=(4, 4))
    fig = plt.figure(figsize=(20,20))
    axarr = fig.subplots(4,4)
    for i in range(4):
        for j in range(4):
            axarr[i,j].imshow(training_images[imgs[i,j]])
            axarr[i,j].grid(False)

plot_data()

In [ ]:
training_images=training_images / 255.0
training_images=training_images.reshape(-1, 28, 28, 1)
testing_images=testing_images/255.0
testing_images=testing_images.reshape(-1, 28, 28, 1)


In [ ]:
model_old = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(25, activation='softmax')
])
model_old.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_old.summary()

In [ ]:
history_old = model_old.fit(training_images, training_labels, epochs=5)

In [ ]:
test_loss_old = model_old.evaluate(testing_images, testing_labels)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(25, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(training_images, training_labels, epochs=5)

In [ ]:
test_loss = model.evaluate(testing_images, testing_labels)

In [ ]:
print(testing_labels[:100])
def visualize_filters(first_img, second_img, third_img, conv_number):
    fig = plt.figure(figsize=(15,10))
    axarr = fig.subplots(3,4)
    from tensorflow.keras import models
    layer_outputs = [layer.output for layer in model.layers]
    activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)
    for x in range(0,4):
        f1 = activation_model.predict(testing_images[first_img].reshape(1, 28, 28, 1))[x]
        axarr[0,x].imshow(f1[0, : , :, conv_number], cmap='inferno')
        axarr[0,x].grid(False)
        f2 = activation_model.predict(testing_images[second_img].reshape(1, 28, 28, 1))[x]
        axarr[1,x].imshow(f2[0, : , :, conv_number], cmap='inferno')
        axarr[1,x].grid(False)
        f3 = activation_model.predict(testing_images[third_img].reshape(1, 28, 28, 1))[x]
        axarr[2,x].imshow(f3[0, : , :, conv_number], cmap='inferno')
        axarr[2,x].grid(False)

In [ ]:
visualize_filters(9, 14, 1, 31)